In [1]:
from config import *
import pandas as pd
import requests
import json
import time
from datetime import datetime

In [2]:
# underlying stock ticker symbol
ticker = 'SPY'

# expiration date of desired option where last number '0' is the days until expiration
exp_date = '2023-01-27:0'

# strike price of desired option in str
strike_price = '405.0'

# sets the desired net delta. net delta = number of share + option delta
target_net_delta = -2.0

# the close time in minutes after midnight, so 810 is 1:30pm local time
close_time = 810

# sets time interval (in seconds) for bot to check position and make adjustments in seconds
time_interval = 10

In [3]:
# Define TDAmeritrade Endpoint and Payload
tda_endpoint = 'https://api.tdameritrade.com/v1/marketdata/chains'
tda_payload = {'apikey': TDA_ID,
               'symbol': ticker,
               'contractType': 'CALL',
               'includeQuotes': 'TRUE',
               'strategy': 'SINGLE',
               'expMonth': 'ALL',
               "isDelayed": 'true'}

# define alpaca endpoint and url
HEADERS = {'APCA-API-KEY-ID': ALPACA_KEY_ID, 'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY}
alpaca_endpoint = 'https://paper-api.alpaca.markets'
account_url = '{}/v2/account'.format(alpaca_endpoint)
orders_url = '{}/v2/orders'.format(alpaca_endpoint)
positions_url = '{}/v2/positions'.format(alpaca_endpoint)
portfolio_url = '{}/v2/account/portfolio/history'.format(alpaca_endpoint)

In [4]:
# this function will get the profit/loss on the stock position in alpaca
def get_pnl():
    data = {
        'period': '1D',
        'timeframe': '1D'
    }
    r = requests.get(portfolio_url, headers=HEADERS)
    return json.loads(r.content)


# returns the number of shares currently open
def num_of_shares(ticker):
    r = requests.get(positions_url + '/{}'.format(ticker), headers=HEADERS)
    return json.loads(r.content)


# puts in an order to buy/sell shares. side='buy' or 'sell'
def create_order(ticker, qty, side): 
    data = {
        'symbol': ticker,
        'qty': qty,
        'side': side,
        'type': 'market',
        'time_in_force': 'day'
    }
    r = requests.post(orders_url, json=data, headers=HEADERS)
    

# returns the delta of the short call
def get_call_delta(tda_data, exp_date, strike_price):
    return tda_data['callExpDateMap'][exp_date][strike_price][0]['delta'] * -100


# closes all alpaca positions
def close_all_positions():
    r = requests.delete(positions_url, headers=HEADERS)

In [5]:
# this returns how much you sold the option for in nominal dollars
r = requests.get(url=tda_endpoint, params=tda_payload)
tda_data = json.loads(r.content)
option_sell_price = tda_data['callExpDateMap'][exp_date][strike_price][0]['bid'] * 100

In [6]:
while (datetime.now().hour * 60 + datetime.now().minute) < close_time: # bot keeps running until 1:30pm local time at which point all positions will be close
    try:
        stock_delta = float(num_of_shares(ticker)['qty'])
    except:
        stock_delta = 0
    
    # gets the option delta and net_delta
    try:
        # get current call data
        r = requests.get(url=tda_endpoint, params=tda_payload)
        tda_data = json.loads(r.content)
        short_call_delta = get_call_delta(tda_data, exp_date, strike_price)
        
        # find net delta
        current_net_delta = round(stock_delta + short_call_delta, 0)
        
        # put in buy/sell orders based on net delta
        if current_net_delta < target_net_delta:
            create_order(ticker, (target_net_delta - current_net_delta), 'buy')
        elif current_net_delta > target_net_delta:
            create_order(ticker, (current_net_delta - target_net_delta), 'sell')

    
        option_pnl = round(option_sell_price - (tda_data['callExpDateMap'][exp_date][strike_price][0]['ask'] * 100), 2)
        stock_pnl = get_pnl()['profit_loss'][-1]

        print('open shares: ', stock_delta)
        print('option delta: ', round(short_call_delta, 1))
        print('net delta: ', current_net_delta)
        print('option profit/loss: ', option_pnl)
        print('stock profit/loss: ', stock_pnl)
        print('net profit/loss: ', round(stock_pnl + option_pnl, 2))
        print(datetime.now().replace(microsecond=0), '\n')
        print()
    except Exception as error:
        print(type(error))

    time.sleep(time_interval) 

# after market closes, all alpaca positions will be closed
close_all_positions()

open shares:  0
option delta:  -72.7
net delta:  -73.0
option profit/loss:  -2.0
stock profit/loss:  0
net profit/loss:  -2.0
2023-01-27 12:30:51 


open shares:  71.0
option delta:  -72.8
net delta:  -2.0
option profit/loss:  -7.0
stock profit/loss:  3.55
net profit/loss:  -3.45
2023-01-27 12:31:02 


open shares:  71.0
option delta:  -73.8
net delta:  -3.0
option profit/loss:  -6.0
stock profit/loss:  3.55
net profit/loss:  -2.45
2023-01-27 12:31:14 


open shares:  72.0
option delta:  -73.8
net delta:  -2.0
option profit/loss:  -6.0
stock profit/loss:  3.54
net profit/loss:  -2.46
2023-01-27 12:31:26 


open shares:  72.0
option delta:  -73.8
net delta:  -2.0
option profit/loss:  -6.0
stock profit/loss:  2.82
net profit/loss:  -3.18
2023-01-27 12:31:37 


open shares:  72.0
option delta:  -73.9
net delta:  -2.0
option profit/loss:  -6.0
stock profit/loss:  4.26
net profit/loss:  -1.74
2023-01-27 12:31:48 


open shares:  72.0
option delta:  -73.6
net delta:  -2.0
option profit/loss:

open shares:  75.0
option delta:  -75.9
net delta:  -1.0
option profit/loss:  -9.0
stock profit/loss:  9.43
net profit/loss:  0.43
2023-01-27 12:41:00 


open shares:  74.0
option delta:  -76.1
net delta:  -2.0
option profit/loss:  -8.0
stock profit/loss:  7.21
net profit/loss:  -0.79
2023-01-27 12:41:12 


open shares:  74.0
option delta:  -75.2
net delta:  -1.0
option profit/loss:  -4.0
stock profit/loss:  4.99
net profit/loss:  0.99
2023-01-27 12:41:23 


open shares:  73.0
option delta:  -74.5
net delta:  -2.0
option profit/loss:  1.0
stock profit/loss:  0.59
net profit/loss:  1.59
2023-01-27 12:41:34 


open shares:  73.0
option delta:  -73.8
net delta:  -1.0
option profit/loss:  6.0
stock profit/loss:  -5.25
net profit/loss:  0.75
2023-01-27 12:41:46 


open shares:  72.0
option delta:  -73.9
net delta:  -2.0
option profit/loss:  2.0
stock profit/loss:  -3.1
net profit/loss:  -1.1
2023-01-27 12:41:58 


open shares:  72.0
option delta:  -74.0
net delta:  -2.0
option profit/loss: 

open shares:  73.0
option delta:  -75.4
net delta:  -2.0
option profit/loss:  -3.0
stock profit/loss:  4.54
net profit/loss:  1.54
2023-01-27 12:51:13 


open shares:  73.0
option delta:  -75.2
net delta:  -2.0
option profit/loss:  -5.0
stock profit/loss:  3.81
net profit/loss:  -1.19
2023-01-27 12:51:24 


open shares:  73.0
option delta:  -75.4
net delta:  -2.0
option profit/loss:  -3.0
stock profit/loss:  0.89
net profit/loss:  -2.11
2023-01-27 12:51:36 


open shares:  73.0
option delta:  -75.1
net delta:  -2.0
option profit/loss:  -2.0
stock profit/loss:  0.89
net profit/loss:  -1.11
2023-01-27 12:51:47 


open shares:  73.0
option delta:  -75.4
net delta:  -2.0
option profit/loss:  -8.0
stock profit/loss:  4.54
net profit/loss:  -3.46
2023-01-27 12:51:59 


open shares:  73.0
option delta:  -76.0
net delta:  -3.0
option profit/loss:  -6.0
stock profit/loss:  5.27
net profit/loss:  -0.73
2023-01-27 12:52:10 


open shares:  74.0
option delta:  -76.3
net delta:  -2.0
option profit/

open shares:  79.0
option delta:  -80.8
net delta:  -2.0
option profit/loss:  -28.0
stock profit/loss:  28.95
net profit/loss:  0.95
2023-01-27 13:01:23 


open shares:  79.0
option delta:  -84.7
net delta:  -6.0
option profit/loss:  -46.0
stock profit/loss:  43.17
net profit/loss:  -2.83
2023-01-27 13:01:35 


open shares:  83.0
option delta:  -82.5
net delta:  0.0
option profit/loss:  -38.0
stock profit/loss:  35.7
net profit/loss:  -2.3
2023-01-27 13:01:46 


open shares:  81.0
option delta:  -82.9
net delta:  -2.0
option profit/loss:  -40.0
stock profit/loss:  38.13
net profit/loss:  -1.87
2023-01-27 13:01:57 


open shares:  81.0
option delta:  -81.0
net delta:  0.0
option profit/loss:  -22.0
stock profit/loss:  20.31
net profit/loss:  -1.69
2023-01-27 13:02:09 


open shares:  79.0
option delta:  -78.7
net delta:  0.0
option profit/loss:  -20.0
stock profit/loss:  17.92
net profit/loss:  -2.08
2023-01-27 13:02:21 


open shares:  77.0
option delta:  -79.0
net delta:  -2.0
option 

open shares:  78.0
option delta:  -80.8
net delta:  -3.0
option profit/loss:  -26.0
stock profit/loss:  22.82
net profit/loss:  -3.18
2023-01-27 13:11:35 


open shares:  79.0
option delta:  -81.9
net delta:  -3.0
option profit/loss:  -28.0
stock profit/loss:  26.78
net profit/loss:  -1.22
2023-01-27 13:11:46 


open shares:  80.0
option delta:  -82.1
net delta:  -2.0
option profit/loss:  -27.0
stock profit/loss:  25.97
net profit/loss:  -1.03
2023-01-27 13:11:58 


open shares:  80.0
option delta:  -82.0
net delta:  -2.0
option profit/loss:  -28.0
stock profit/loss:  26.77
net profit/loss:  -1.23
2023-01-27 13:12:09 


open shares:  80.0
option delta:  -81.9
net delta:  -2.0
option profit/loss:  -28.0
stock profit/loss:  25.17
net profit/loss:  -2.83
2023-01-27 13:12:20 


open shares:  80.0
option delta:  -82.0
net delta:  -2.0
option profit/loss:  -30.0
stock profit/loss:  26.77
net profit/loss:  -3.23
2023-01-27 13:12:32 


open shares:  80.0
option delta:  -81.1
net delta:  -1.0
o

open shares:  86.0
option delta:  -87.9
net delta:  -2.0
option profit/loss:  -61.0
stock profit/loss:  58.67
net profit/loss:  -2.33
2023-01-27 13:21:43 


open shares:  86.0
option delta:  -87.0
net delta:  -1.0
option profit/loss:  -57.0
stock profit/loss:  54.37
net profit/loss:  -2.63
2023-01-27 13:21:55 


open shares:  85.0
option delta:  -86.9
net delta:  -2.0
option profit/loss:  -60.0
stock profit/loss:  56.06
net profit/loss:  -3.94
2023-01-27 13:22:06 


open shares:  85.0
option delta:  -86.9
net delta:  -2.0
option profit/loss:  -57.0
stock profit/loss:  51.81
net profit/loss:  -5.19
2023-01-27 13:22:18 


open shares:  85.0
option delta:  -87.3
net delta:  -2.0
option profit/loss:  -59.0
stock profit/loss:  56.91
net profit/loss:  -2.09
2023-01-27 13:22:29 


open shares:  85.0
option delta:  -87.4
net delta:  -2.0
option profit/loss:  -63.0
stock profit/loss:  61.16
net profit/loss:  -1.84
2023-01-27 13:22:41 


open shares:  85.0
option delta:  -87.2
net delta:  -2.0
o